# 11. 내부 클래스 + 람다식 + 스트림 


## 내부 클래스
- 클래스 내부에 구현된 클래스(중첩된 클래스)
- 클래스 내부에서 사용하기 위해 선언하구 구현하는 클래스 
- 주로 외부 클래스 생성자에서 내부 클래스를 생성
- 인스턴스 내부 클래스, 정적 내부 클래스, 지역 내부 클래스, 익명 내부 클래스 등으로 나눠짐
- 익명 내부 클래스가 가장 많이 쓰인다
- static은 인스턴스 생성과 상관 없이 쓸 수 있다 어짜피 클래스로 불리기 때문에

### 인스턴스 내부 클래스, 정적 내부 클래스

In [1]:
class OutClass{
    private int num = 10;
    private static int sNum = 20;
    private InClass inClass;
    
    public OutClass() {
        // 바깥 클래스 생성자로 인스턴스 생성하면 새로운 안 클래스 인스턴스 생성
        inClass = new InClass();
    }
    
    // static으로 선언하면 바깥 클래스 생성자에서 못쓴다
    // 정적 클래스는 바깥 클래스와 아무 상관 없이 그냥 쓸수 있음
    // 인스턴스 내부 클래스
    class InClass{
        int iNum = 100;
        
        void inTest() {
            System.out.println(num);
            System.out.println(sNum);
        }
    }
    
    public void usingInner() {
        inClass.inTest();
    }
    
    // 스태틱 내부 클래스의 경우 스태틱 메소드도 생성 가능함
    // 정적 내부 클래스
    static class InStaticClass{
        int inNum = 100;
        static int sInNum = 200;
        
        void inTest() {
            System.out.println(inNum);
            System.out.println(sInNum);
            System.out.println(sNum);
        }
        
        // 요 정적 클래스의 생성과 상관없이 접근할 수 있는 클래스이므로
        // inNum에 접근할 수 없음
        // static으로 선언한 다른 변수는 접근 가능함
        static void sTest() {
            System.out.println(sInNum);
            System.out.println(sNum);
        }
    }
}

In [2]:
OutClass outClass = new OutClass();
outClass.usingInner();

10
20


In [3]:
// 참조변수 사용해서 안에서 선언한 클래스 꺼내쓸 수 있다
// Private으로 만들면 안되긴함

OutClass.InClass myInClass = outClass.new InClass();
myInClass.inTest();

10
20


### 지역 내부 클래스, 익명 내부 클래스

In [4]:
class Outer{
    int outNum = 100;
    static int sNum = 200;
    
    // 메소드 안에서의 클래스 선언
    // 인자 받은 상태이며 지역변수 값은 modify가능
    // 지역변수 유효 => 메서드가 호출되어 끝날때까지 유효
    // 메소드 안의 내부 클래스는 상위 메소드의 변수에 접근할 수 없다(변수의 상수화) 밖이 더 오래 살아있기 때문에???정확히 잘 모르겟음
    Runnable getRunnable(int i) {
        int num = 100;
        // 바로 클래스를 반환해버릴수도 있음 => return new Runnable(int i){}
        // 이게 익명 내부 클래스(이름이 없는 익명 클래스)
        class MyRunnable implements Runnable{
            public void run() {
                // 상위 클래스에 접근 가능
                System.out.println(num);
                System.out.println(i);
                System.out.println(outNum);
                System.out.println(Outer.sNum);
            }
        }
        return new MyRunnable();
    }
}

In [5]:
Outer outer = new Outer();
Runnable runnable = outer.getRunnable(50);

runnable.run();

100
50
100
200


In [6]:
// 냅다 new 써서 클래스 만들어버리기 - 익명 클래스
// 단 하나의 인터페이스, 단 하나의 추상클래스인 경우는 클래스 이름 없이 뉴 키워드 생성해서
// 이런식으로 구현할 수 있다

Runnable runner = new Runnable(){
    public void run() {
        System.out.println("test");
    }
};
// 익명 클래스 선언 끝에 세미콜론 붙여야함

runner.run();

test


## 람다식

- 자바에서 함수형 프로그래밍을 구현하는 방식
- 자바의 동적타이핑
- 클래스를 생성하지 않고 함수의 호출만으로 기능을 수행
- 함수형 인터페이스를 선언함
- since 자바8
- 함수를 변수처럼 사용할 수 있음(함수의 표현식이 변수에 대입이 되서 사용되는)

### 함수형 프로그래밍 
- 순수함수를 구현하고 호출
- 사이드 이펙트 발생하지 않음
- 안정적인 프로그래밍 방식


In [7]:
// 함수형 인터페이스
// 타입스크립트 함수 시그니처를 생각나게한다

@FunctionalInterface
public interface MyMaxNumber {
    int getMaxNumber(int x, int y);
}

In [8]:
MyMaxNumber max = (x,y) -> (x >= y)?x:y;
System.out.println(max.getMaxNumber(10,20));

20


In [9]:
@FunctionalInterface
public interface StringConcat {
    public void makeString(String s1, String s2);
}

In [10]:
public class StringConImp1 implements StringConcat {
    public void makeString(String s1, String s2) {
        System.out.println(s1 + "," + s2);
    }
}

In [11]:
// 일반적인 방법

StringConImp1 imp1 = new StringConImp1();
imp1.makeString("hello", "world");

hello,world


In [12]:
// 람다식 사용
// 클래스 구현 필요 없이 인터페이스 만으로 바로 인스턴스를 만들 수 있음
// 내부에 익명 객체를 생성해서 만드는 것
// 함수를 변수처럼 사용하는 느낌으로다가

StringConcat concat = (s,v) -> System.out.println(s+","+v);
concat.makeString("hello", "world");

hello,world


In [13]:
interface PrintString {
    void showString(String str);
}

In [14]:
PrintString lambdaStr = s -> System.out.println(s);
lambdaStr.showString("Test");

Test


In [15]:
public static void showMyString(PrintString p) {
    p.showString("Test2");
}

In [16]:
// 이렇게 파라미터로도 넘길 수 있게 되서 함수형 프로그래밍이 가능하다

showMyString(lambdaStr);

Test2


In [17]:
// 특이하게 람다식 사용하면 파라미터를 명시해주지 않아도 괜찮다

public static PrintString returnString() {
    return s -> System.out.println(s + "!!!");
}

In [18]:
PrintString test = returnString();
test.showString("Test3")

Test3!!!
